In [50]:
import scipy.io
import numpy as np

In [51]:
mat_data = scipy.io.loadmat('./data_preprocessed_matlab/s01.mat') 
original_data = mat_data['data']
original_label = mat_data['labels']

# (40, 40, 8064) 40试验，40通道（前32为eeg），63s（前3秒没用） x 128Hz = 8064
print("original_data形状：", original_data.shape)
# (40, 4) 40试验，4个评价指标
print("original_label形状：", original_label.shape)

original_data形状： (40, 40, 8064)
original_label形状： (40, 4)


In [52]:
'''
    处理eeg，从(40, 40, 8064)变成(800, 32, 384)
'''
# 前32为eeg，去除前3秒
sliced_data = original_data[:, :32, 384:] 
print("sliced_data形状：", sliced_data.shape) # (40, 32, 7680)

# 将数据切片成 3 秒一段
segment_size = 3 * 128
num_segments = sliced_data.shape[2] // segment_size # 7680/3/128 = 20
segments_array = np.zeros((sliced_data.shape[0], sliced_data.shape[1], num_segments, segment_size))

# 将数据按照每 384 个元素一段切开，并存储到 segments_array 中
for i in range(num_segments):
    start_index = i * segment_size
    end_index = (i + 1) * segment_size
    segments_array[:, :, i, :] = sliced_data[:, :, start_index:end_index]
# (40, 32, 20, 384)
print("segments_array形状：", segments_array.shape)

# 使用 transpose 函数交换第二和第三维度
reshaped_segments_array = segments_array.transpose(2, 0, 1, 3)
# (20, 40, 32, 384)
print("重新排列后的数组形状：", reshaped_segments_array.shape)

# 使用 reshape 函数合并前两个维度
merged_segments_array = reshaped_segments_array.reshape(-1, reshaped_segments_array.shape[2], reshaped_segments_array.shape[3])
# (800, 32, 384)
print("合并后的数组形状：", merged_segments_array.shape)
eeg_data = merged_segments_array

sliced_data形状： (40, 32, 7680)
segments_array形状： (40, 32, 20, 384)
重新排列后的数组形状： (20, 40, 32, 384)
合并后的数组形状： (800, 32, 384)


In [53]:
'''
    处理labels，从(40, 4)变成(800,)
'''
valence = original_label[:,0]
arousal = original_label[:,1]
# HAHV--1, LAHV--2, LALV--3, HALV--4 完全按照象限来分
VA_labels = np.where((valence > 5) & (arousal > 5), 1,
          np.where((valence >= 5) & (arousal < 5), 2,
            np.where((valence < 5) & (arousal < 5), 3, 4)))
print("V:", valence)
print("A:", arousal)
print(VA_labels)

# 将 VA_labels 从 40 扩展到 800，VA_labels中的一个数复制20次
expanded_VA_labels = np.repeat(VA_labels, num_segments)
print(expanded_VA_labels.shape) # (800,)
labels = expanded_VA_labels 

V: [7.71 8.1  8.58 4.94 6.96 8.27 7.44 7.32 4.04 1.99 2.99 2.71 1.95 4.18
 3.17 6.81 2.46 7.23 7.17 8.26 9.   7.09 8.15 7.04 8.86 7.28 7.35 3.88
 1.36 2.08 3.03 2.28 3.81 2.28 2.06 2.9  2.31 3.33 3.24 5.1 ]
A: [7.6  7.31 7.54 6.01 3.92 3.92 3.73 2.55 3.29 4.86 2.36 2.77 3.12 2.24
 8.08 7.44 6.91 7.15 8.   7.91 7.95 2.08 3.01 7.09 7.21 7.27 6.95 3.35
 2.27 2.99 8.14 8.   3.85 7.09 8.15 6.92 6.88 7.18 6.18 7.12]
[1 1 1 4 2 2 2 2 3 3 3 3 3 3 4 1 4 1 1 1 1 2 2 1 1 1 1 3 3 3 4 4 3 4 4 4 4
 4 4 1]
(800,)


In [54]:
'''
    保存数据
'''
np.save('./EEGData/s01_eeg.npy', eeg_data) # (800, 32, 384)
np.save('./EEGData/s01_labels.npy', labels) # (800,)

In [59]:
import pandas as pd
import mne
from mne import io
from mne.datasets import sample
from mne.preprocessing import ICA
import os

sfreq = 128  # 采样率为128Hz
channels = 32
samples = 384
num_trials = 800
ch_names = ['Fp1', 'AF3', 'F7', 'F3', 'FC1', 'FC5', 'T7', 'C3', 'CP1', 'CP5', 'P7', 'P3', 
            'Pz', 'PO3', 'O1', 'Oz', 'O2', 'PO4', 'P4', 'P8', 'CP6', 'CP2', 'C4', 'T8', 
            'FC6', 'FC2', 'F4', 'F8', 'AF4', 'FP2', 'Fz', 'Cz']
ch_types = ['eeg'] * channels

# eeg_data_file = os.path.join(eeg_data_folder, 's01_eeg.npy')
# label_file = os.path.join(eeg_data_folder, 's01_labels.npy')
# eeg_data = np.load(eeg_data_file)
# labels = np.load(label_file)

data_list = []
# 创建MNE的Raw对象
info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sfreq)
eeg_data_segments = np.split(eeg_data, num_trials, axis=0)
for segment in eeg_data_segments:
    segment_2d = segment.reshape(-1, channels).T
    print("wzt segment_2d", segment_2d.shape) # (32, 384)
    # 创建Raw对象
    raw = mne.io.RawArray(segment_2d, info=info)
    
    # 滤波
    raw.filter(l_freq=1.0, h_freq=50.0)
    
    # 创建ICA对象并拟合数据
    ica = ICA(n_components=channels, random_state=0, max_iter=1000)  # 调整参数
    ica.fit(raw)
    
    # 应用ICA滤波
    ica.exclude = []  # 将排除的独立成分列表设置为空
    ica.apply(raw)
    
    # 将滤波后的数据添加到列表
    data_list.append(raw.get_data().T)

# 将数据列表转换为3D数组
eeg_data_filtered = np.array(data_list)
print("wzt eeg_data_filtered", eeg_data_filtered.shape)

wzt segment_2d (32, 384)
Creating RawArray with float64 data, n_channels=32, n_times=384
    Range : 0 ... 383 =      0.000 ...     2.992 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 423 samples (3.305 s)

Fitting ICA to data using 32 channels (please be patient, this may take a while)
Selecting by number: 32 components
Fitting ICA took 0.1s.
Applying ICA to Raw instance
    Transforming to ICA space (32 components)
    Zeroing out 0 ICA components
    Projecting back using 32 PCA component

/var/folders/29/nx05yj9x3db48wln_03fc6tc0000gn/T/ipykernel_19586/3326501990.py:33: RuntimeWarning: filter_length (423) is longer than the signal (384), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(l_freq=1.0, h_freq=50.0)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
/var/folders/29/nx05yj9x3db48wln_03fc6tc0000gn/T/ipykernel_19586/3326501990.py:37: RuntimeWarning: Using n_components=32 (resulting in n_components_=32) may lead to an unstable mixing matrix estimation because the ratio between the largest (9.6) and smallest (1.3e-13) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 27
  ica.fit(raw)
/var/folders/29/nx05yj9x3db48wln_03fc6tc0000gn/T/ipykernel_19586/3326501990.py:33: RuntimeWarning: filter_length (423) is longer than the signal (384), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(l_freq=1.0, h_freq=50.0)
[Parallel(n_jobs=1)]: Done  17 tasks      | 

Fitting ICA took 0.0s.
Applying ICA to Raw instance
    Transforming to ICA space (32 components)
    Zeroing out 0 ICA components
    Projecting back using 32 PCA components
wzt segment_2d (32, 384)
Creating RawArray with float64 data, n_channels=32, n_times=384
    Range : 0 ... 383 =      0.000 ...     2.992 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 423 samples (3.305 s)

Fitting ICA to data using 32 channels (please be patient, this may take a while)
Selecting by number: 32 component

/var/folders/29/nx05yj9x3db48wln_03fc6tc0000gn/T/ipykernel_19586/3326501990.py:37: RuntimeWarning: Using n_components=32 (resulting in n_components_=32) may lead to an unstable mixing matrix estimation because the ratio between the largest (7.9) and smallest (1.3e-13) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 27
  ica.fit(raw)
/var/folders/29/nx05yj9x3db48wln_03fc6tc0000gn/T/ipykernel_19586/3326501990.py:33: RuntimeWarning: filter_length (423) is longer than the signal (384), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(l_freq=1.0, h_freq=50.0)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
/var/folders/29/nx05yj9x3db48wln_03fc6tc0000gn/T/ipykernel_19586/3326501990.py:37: RuntimeWarning: Using n_components=32 (resulting in n_components_=32) may lead to an unstable mixing matrix estimation because the ratio between the largest (12) and smallest (1.2e-13) variances is too large (> 1e6); cons

Fitting ICA took 0.1s.
Applying ICA to Raw instance
    Transforming to ICA space (32 components)
    Zeroing out 0 ICA components
    Projecting back using 32 PCA components
wzt segment_2d (32, 384)
Creating RawArray with float64 data, n_channels=32, n_times=384
    Range : 0 ... 383 =      0.000 ...     2.992 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 423 samples (3.305 s)

Fitting ICA to data using 32 channels (please be patient, this may take a while)
Selecting by number: 32 component

...

    Zeroing out 0 ICA components
    Projecting back using 32 PCA components
wzt segment_2d (32, 384)
Creating RawArray with float64 data, n_channels=32, n_times=384
    Range : 0 ... 383 =      0.000 ...     2.992 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 423 samples (3.305 s)

Fitting ICA to data using 32 channels (please be patient, this may take a while)
Selecting by number: 32 components
Fitting ICA took 0.1s.
Applying ICA to Raw instance
    Transforming to ICA space (32 components

/var/folders/29/nx05yj9x3db48wln_03fc6tc0000gn/T/ipykernel_19586/3326501990.py:33: RuntimeWarning: filter_length (423) is longer than the signal (384), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(l_freq=1.0, h_freq=50.0)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
/var/folders/29/nx05yj9x3db48wln_03fc6tc0000gn/T/ipykernel_19586/3326501990.py:37: RuntimeWarning: Using n_components=32 (resulting in n_components_=32) may lead to an unstable mixing matrix estimation because the ratio between the largest (6) and smallest (2.7e-13) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 27
  ica.fit(raw)
/var/folders/29/nx05yj9x3db48wln_03fc6tc0000gn/T/ipykernel_19586/3326501990.py:33: RuntimeWarning: filter_length (423) is longer than the signal (384), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(l_freq=1.0, h_freq=50.0)
[Parallel(n_jobs=1)]: Done  17 tasks      | el

Fitting ICA took 0.1s.
Applying ICA to Raw instance
    Transforming to ICA space (32 components)
    Zeroing out 0 ICA components
    Projecting back using 32 PCA components
wzt segment_2d (32, 384)
Creating RawArray with float64 data, n_channels=32, n_times=384
    Range : 0 ... 383 =      0.000 ...     2.992 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 423 samples (3.305 s)

Fitting ICA to data using 32 channels (please be patient, this may take a while)
Selecting by number: 32 component

/var/folders/29/nx05yj9x3db48wln_03fc6tc0000gn/T/ipykernel_19586/3326501990.py:37: RuntimeWarning: Using n_components=32 (resulting in n_components_=32) may lead to an unstable mixing matrix estimation because the ratio between the largest (8.9) and smallest (2.1e-13) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 27
  ica.fit(raw)
/var/folders/29/nx05yj9x3db48wln_03fc6tc0000gn/T/ipykernel_19586/3326501990.py:33: RuntimeWarning: filter_length (423) is longer than the signal (384), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(l_freq=1.0, h_freq=50.0)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
/var/folders/29/nx05yj9x3db48wln_03fc6tc0000gn/T/ipykernel_19586/3326501990.py:37: RuntimeWarning: Using n_components=32 (resulting in n_components_=32) may lead to an unstable mixing matrix estimation because the ratio between the largest (7.8) and smallest (3.1e-13) variances is too large (> 1e6); con

    Transforming to ICA space (32 components)
    Zeroing out 0 ICA components
    Projecting back using 32 PCA components
wzt eeg_data_filtered (800, 384, 32)
